## Анализ данных для агентства недвижимости


### Цели и задачи проекта

**Цель:**  Выявить и проанализировать наиболее перспективные сегменты рынка жилой недвижимости Санкт-Петербурга и Ленинградской области на основе анализа объявлений о продаже, чтобы определить приоритетные направления работы Агентства.

### Описание данных

Представлены данные архива сервиса Яндекс Недвижимость: объявления о продаже квартир в Санкт-Петербурге и Ленинградской области за несколько лет (с конца ноября 2014 года по начало мая 2019 года). 

Схема `real_estate` включает следующие таблицы:

### Структура базы данных

**Таблица `advertisement`** (объявления):
- `id` — идентификатор объявления (первичный ключ)
- `first_day_exposition` — дата подачи объявления
- `days_exposition` — длительность нахождения на сайте (дни)
- `last_price` — стоимость квартиры (руб)

**Таблица `flats`** (квартиры):
- `id` — идентификатор квартиры (первичный ключ, связан с `advertisement.id`)
- `city_id` — идентификатор города (внешний ключ)
- `type_id` — идентификатор типа населённого пункта (внешний ключ)
- `total_area` — общая площадь (м²)
- `rooms` — число комнат
- `ceiling_height` — высота потолков (м)
- `floors_total` — этажность дома
- `living_area` — жилая площадь (м²)
- `floor` — этаж квартиры
- `is_apartment` — является апартаментами (1/0)
- `open_plan` — открытая планировка (1/0)
- `kitchen_area` — площадь кухни (м²)
- `balcony` — количество балконов
- `airports_nearest` — расстояние до аэропорта (м)
- `parks_around3000` — число парков в радиусе 3 км
- `ponds_around3000` — число водоёмов в радиусе 3 км

**Таблица `city`** (города):
- `city_id` — идентификатор населённого пункта (первичный ключ)
- `city` — название населённого пункта

**Таблица `type`** (типы населённых пунктов):
- `type_id` — идентификатор типа (первичный ключ)
- `type` — название типа населённого пункта

#### План проекта

[1. Анализ времени активности объявлений](#1-bullet)

[2. Анализ сезонности объявлений](#2-bullet)

[3. Анализ рынка недвижимости Ленобласти](#3-bullet)

[4. Общие выводы и рекомендации](#4-bullet)

In [1]:
pip install psycopg2-binary 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext sql

In [2]:
import psycopg2
import pandas as pd


In [4]:
#Подключаем базу данных
DB_HOST = 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net'
DB_PORT = '6432'
DB_NAME = 'data-analyst-estate'
DB_USER = 'praktikum_student'
DB_PASS = 'Sdf4$2;d-d30pp'

conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)
import warnings
warnings.filterwarnings('ignore')

<a class="anchor" id="1-bullet"></a>
#### 1. Анализ времени активности объявлений

Чтобы спланировать эффективную бизнес-стратегию на рынке недвижимости, необходимо понять, какие типы квартир Санкт-Петербурга и городов Ленинградской области продаются быстро, то есть объявления о них быстро снимаются, а какие — долго остаются активными на площадках.

При первичном знакомстве с данными обнаружены аномальные и нереальные значения, отфильтруем аномальные значения и оставим id объявлений с корректными значениями с помощью подзапроса:

```sql
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),

-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
)

-- Выведем объявления без выбросов:
SELECT *
FROM real_estate.flats
WHERE id IN (SELECT * FROM filtered_id);

Всего отфильтровано 19% данных

Разделим объявления на категории по количеству дней активности: 
- 1-30 days — около одного месяца;
- 31-90 days — от одного до трёх месяцев;
- 91-180 days — от трёх месяцев до полугода;
- 181+ days — более полугода

In [5]:
# SQL-запрос
sql_query = """
WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats     
),

-- Найдём id объявлений, которые не содержат выбросы:
filtered_id AS(
    SELECT id
    FROM real_estate.flats  
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
),

-- Категоризируем объявления по времени активности и отфильтруем объявления только по городам:
by_city AS (
    SELECT
        CASE 
            WHEN city = 'Санкт-Петербург' THEN 'Санкт-Петербург'
            ELSE 'ЛенОбл'
        END AS city_category,
        CASE 
            WHEN days_exposition > 0 AND days_exposition <= 30 THEN 'sold_in_month'
            WHEN days_exposition > 30 AND days_exposition <= 90 THEN 'sold_in_three_months'
            WHEN days_exposition > 90 AND days_exposition <= 180 THEN 'sold_in_six_months'
            WHEN days_exposition > 180 THEN 'sold_in_more_then_six'
        END AS act_category,
        last_price/total_area AS square,
        total_area,
        rooms,
        balcony,
        "floor",
        id
    FROM real_estate.flats 
    JOIN real_estate.city USING (city_id)
    JOIN real_estate.TYPE USING (type_id)
    JOIN real_estate.advertisement USING(id)
    WHERE "type" = 'город'
)

SELECT 
    city_category,
    act_category,
    ROUND(AVG(square)::NUMERIC, 2) AS avg_sum_per_square,
    ROUND(AVG(total_area)::NUMERIC, 2) AS avg_total_area,
    PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY rooms) AS mediana_rooms,
    PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY balcony) AS mediana_balcony,
    PERCENTILE_DISC(0.5) WITHIN GROUP (ORDER BY "floor") AS mediana_floor,
    COUNT(id) AS total_ads
FROM by_city
WHERE id IN (SELECT * FROM filtered_id) 
  AND act_category IS NOT NULL
GROUP BY city_category, act_category
ORDER BY city_category
"""

In [6]:
# Выполнение запроса и сохранение результатов в DataFrame
df_result = pd.read_sql_query(sql_query, conn)
df_result


,city_category,act_category,avg_sum_per_square,avg_total_area,mediana_rooms,mediana_balcony,mediana_floor,total_ads
0,ЛенОбл,sold_in_month,73275.25,48.72,2,1.0,4,397
1,ЛенОбл,sold_in_more_then_six,68297.22,55.41,2,1.0,3,890
2,ЛенОбл,sold_in_six_months,69846.39,51.83,2,1.0,3,556
3,ЛенОбл,sold_in_three_months,67573.43,50.88,2,1.0,3,917
4,Санкт-Петербург,sold_in_month,110568.88,54.38,2,1.0,5,2168
5,Санкт-Петербург,sold_in_more_then_six,115457.22,66.15,2,1.0,5,3581
6,Санкт-Петербург,sold_in_six_months,111938.92,60.55,2,1.0,5,2254
7,Санкт-Петербург,sold_in_three_months,111573.24,56.71,2,1.0,5,3236


### Ключевые выводы анализа

#### 1. Зависимость скорости продажи от площади
- Быстро продающиеся объекты (в течение месяца): квартиры малой площади
  - СПб: 54.38 м²
  - ЛО: 48.72 м²
- Медленно продающиеся объекты (более полугода): квартиры большой площади
  - СПб: 66.15 м²  
  - ЛО: 55.41 м²

**Вывод:** Рынок демонстрирует высокий спрос на компактное жилье и низкую ликвидность крупногабаритных квартир в обоих регионах.

#### 2. Ключевые характеристики ликвидной недвижимости
- Наиболее популярные параметры:
  - 2-комнатные квартиры
  - 1 балкон
  - этажность 3-5 этажей

- В Санкт-Петербурге наблюдается обратная зависимость: чем выше цена за м², тем дольше срок экспозиции объявления.

#### 3. Региональные различия
- Ленинградская область:
  - Более высокая стоимость м² у быстропродающихся объектов
  - Меньший общий метраж
  - Преобладание низкоэтажной застройки

- Санкт-Петербург:
  - Более выраженная зависимость цены от скорости продажи
  - Крупногабаритные квартиры имеют значительно большую площадь (66.15 м² против 55.41 м² в ЛО)

**Общий вывод:** Рынок недвижимости демонстрирует четкую сегментацию по скорости продажи, при этом наблюдаются существенные региональные различия в характеристиках ликвидных объектов между Санкт-Петербургом и Ленинградской областью.

<a class="anchor" id="2-bullet"></a>
#### Анализ сезонности объявлений

Задача выявить сезонные колебания и закономерности на рынке недвижимости Санкт-Петербурга и Ленинградской области для оптимизации бизнес-стратегии и планирования (маркетинга, сроков выхода на рынок)

In [7]:
#SQL-запрос
sql_query2="""WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY last_price) AS last_price_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY last_price) AS last_price_limit_l
    FROM real_estate.flats
    JOIN real_estate.advertisement USING(id)
),

-- Найдём id объявлений, которые не содержат выбросы по площади и цене:
filtered_id AS(
    SELECT id
    FROM real_estate.flats
    JOIN real_estate.type AS t USING(type_id)
    JOIN real_estate.advertisement a USING(id)
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND ((a.last_price < (SELECT last_price_limit_h FROM limits) 
              AND a.last_price > (SELECT last_price_limit_l FROM limits)) 
             OR a.last_price IS NULL)
        AND "type" = 'город'
),

first_day AS (
    SELECT 
        EXTRACT(MONTH FROM first_day_exposition) AS month_first_day,
        ROUND(AVG(last_price/total_area)::NUMERIC, 2) AS avg_meter_cost,
        ROUND(AVG(total_area)::NUMERIC, 2) AS avg_total_area,
        COUNT(id) AS count_ads
    FROM real_estate.advertisement a 
    JOIN real_estate.flats f USING(id)
    WHERE 
        days_exposition IS NOT NULL 
        AND first_day_exposition BETWEEN '2015-01-01' AND '2019-01-01' 
        AND id IN (SELECT * FROM filtered_id)
    GROUP BY month_first_day
),

last_day AS (
    SELECT 
        EXTRACT(MONTH FROM (first_day_exposition + days_exposition * INTERVAL '1 day')::DATE) AS month_last_day,
        ROUND(AVG(last_price/total_area)::NUMERIC, 2) AS avg_meter_cost,
        ROUND(AVG(total_area)::NUMERIC, 2) AS avg_total_area,
        COUNT(id) AS count_ads
    FROM real_estate.advertisement a 
    JOIN real_estate.flats f USING(id)
    WHERE 
        days_exposition IS NOT NULL 
        AND first_day_exposition BETWEEN '2015-01-01' AND '2019-01-01' 
        AND id IN (SELECT * FROM filtered_id)
    GROUP BY month_last_day
) 

SELECT 
    'публикация' AS type,
    *,
    RANK() OVER (ORDER BY count_ads DESC) AS month_rank
FROM first_day

UNION ALL

SELECT 
    'снятие' AS type,
    *,
    RANK() OVER (ORDER BY count_ads DESC) AS month_rank
FROM last_day"""


In [9]:
df_result2 = pd.read_sql(sql_query2, conn)
df_result2

,type,month_first_day,avg_meter_cost,avg_total_area,count_ads,month_rank
0,публикация,11.0,100180.97,57.98,1761,1
1,публикация,10.0,99945.37,57.27,1613,2
2,публикация,2.0,99280.21,59.05,1604,3
3,публикация,9.0,103040.32,58.80,1515,4
4,публикация,6.0,100705.13,57.56,1374,5
5,публикация,8.0,102179.24,56.58,1331,6
6,публикация,3.0,98914.16,58.75,1308,7
7,публикация,7.0,100753.21,58.00,1298,8
8,публикация,4.0,97955.80,59.22,1127,9
9,публикация,12.0,99556.74,56.89,1104,10


### Выводы по сезонности рынка недвижимости

#### 1. Сезонные пики активности
- Публикация объявлений: наибольшая активность наблюдается в осенне-зимний период
  - Ноябрь — 1761 объявлений
  - Октябрь — 1613 объявлений  
  - Февраль — 1604 объявлений

- Снятие объявлений (продажи): пики приходятся на аналогичный период
  - Ноябрь — 1619 сделок
  - Октябрь — 1619 сделок
  - Январь — 1465 сделок

#### 2. Синхронность рыночной активности
**Вывод**: Периоды активной публикации объявлений и повышенной продажи недвижимости частично совпадают:
- Полное совпадение наблюдается в **октябре и ноябре**
- Это свидетельствует о высокой ликвидности в осенний период

#### 3. Влияние сезонности на ценовые параметры
Стоимость квадратного метра:
- Минимальная стоимость при публикации:
  - Апрель — 97 955.80 ₽
  - Март — 98 914.16 ₽

- Максимальная стоимость при публикации:
  - Сентябрь — 103 040.32 ₽

- Период низких цен при продажах:
  - Май-август (97-99 тыс. ₽)

Площадь квартир:
- Сезонность не оказывает существенного влияния на среднюю площадь выставленных и проданных квартир
- Размер жилья остается стабильным параметром в течение года

<a class="anchor" id="3-bullet"></a>
#### Анализ рынка недвижимости Ленобласти
Задача выявить населённые пункты Ленинградской области с наибольшей деловой активностью на рынке недвижимости и определить характеристики наиболее ликвидных объектов в каждом из них.

In [10]:
sql_query3= """
WITH request AS (
    SELECT DISTINCT 
        c.city,
        COUNT(id) OVER(PARTITION BY city) AS amount_ids
    FROM real_estate.flats
    JOIN real_estate.city c USING(city_id) 
    WHERE city != 'Санкт-Петербург'
)
SELECT 
    MIN(amount_ids),
    MAX(amount_ids),
    AVG(amount_ids),
    PERCENTILE_DISC(0.5) WITHIN GROUP(ORDER BY amount_ids) AS mediana,
    PERCENTILE_DISC(0.99) WITHIN GROUP(ORDER BY amount_ids) AS perc_up,
    PERCENTILE_DISC(0.01) WITHIN GROUP(ORDER BY amount_ids) AS perc_down
FROM request;

-- Cделаю отбор топ15 по количеству объявлений.

WITH limits AS (
    SELECT  
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY last_price) AS last_price_limit_h,
        PERCENTILE_DISC(0.01) WITHIN GROUP (ORDER BY last_price) AS last_price_limit_l
    FROM real_estate.flats
    JOIN real_estate.advertisement USING(id)
),

-- Найдём id объявлений, которые не содержат выбросы по площади и цене:
filtered_id AS (
    SELECT id
    FROM real_estate.flats f
    JOIN real_estate.advertisement a USING(id)
    WHERE 
        total_area < (SELECT total_area_limit FROM limits)
        AND ((a.last_price < (SELECT last_price_limit_h FROM limits) 
              AND a.last_price > (SELECT last_price_limit_l FROM limits)) 
             OR a.last_price IS NULL)
),

request AS (
    SELECT DISTINCT 
        c.city,
        COUNT(id) OVER(PARTITION BY city) AS amount_ids
    FROM real_estate.flats
    JOIN real_estate.city c USING(city_id) 
    WHERE city != 'Санкт-Петербург' 
      AND id IN (SELECT * FROM filtered_id)
),

-- Подзапрос ранжирующий населенные пункты по кол-ву объявлений
city_rank AS (
    SELECT *,
        DENSE_RANK() OVER (ORDER BY amount_ids DESC) AS city_rank
    FROM request 
    LIMIT 15
)

-- Основной запрос с отбором топ15 городов по кол-ву объявлений
SELECT 
    city, 
    COUNT(id) AS amount_ids, 
    ROUND((COUNT(id) FILTER (WHERE days_exposition IS NOT NULL)) / (COUNT(id)::NUMERIC), 2) AS selled_share,
    ROUND(AVG(last_price/total_area)::NUMERIC, 2) AS avg_cost_per_square_meter,
    ROUND(AVG(total_area)::NUMERIC, 2) AS avg_total_area, 
    ROUND(COUNT(id) FILTER (WHERE days_exposition < 30) / COUNT(id) FILTER (WHERE days_exposition IS NOT NULL)::NUMERIC, 2) AS share_of_selled_in_month,
    ROUND(COUNT(id) FILTER (WHERE days_exposition >= 30 AND days_exposition < 90) / COUNT(id) FILTER (WHERE days_exposition IS NOT NULL)::NUMERIC, 2) AS share_of_selled_in_three_month,
    ROUND(COUNT(id) FILTER (WHERE days_exposition >= 90 AND days_exposition < 180) / COUNT(id) FILTER (WHERE days_exposition IS NOT NULL)::NUMERIC, 2) AS share_of_selled_in_six_month,
    ROUND(COUNT(id) FILTER (WHERE days_exposition >= 180) / COUNT(id) FILTER (WHERE days_exposition IS NOT NULL)::NUMERIC, 2) AS share_of_selled_in_morethansix
FROM real_estate.flats f 
JOIN real_estate.advertisement a USING(id)
JOIN real_estate.city c USING(city_id)
WHERE city IN (SELECT city FROM city_rank WHERE city_rank <= 15) 
  AND id IN (SELECT * FROM filtered_id)
GROUP BY city
ORDER BY selled_share DESC, amount_ids DESC;"""

In [11]:
df_result3 = pd.read_sql(sql_query3, conn)
df_result3

,city,amount_ids,selled_share,avg_cost_per_square_meter,avg_total_area,share_of_selled_in_month,share_of_selled_in_three_month,share_of_selled_in_six_month,share_of_selled_in_morethansix
0,Мурино,590,0.93,86087.51,44.10,0.21,0.35,0.19,0.25
1,Кудрово,472,0.93,95324.93,46.40,0.19,0.37,0.20,0.25
2,Шушары,440,0.93,78677.36,53.82,0.16,0.33,0.24,0.27
3,Парголово,326,0.92,90332.26,51.06,0.17,0.36,0.20,0.27
4,Колпино,337,0.91,75333.30,52.36,0.22,0.34,0.19,0.25
5,Сестрорецк,182,0.89,101912.34,62.54,0.13,0.30,0.19,0.38
6,Красное Село,176,0.89,72164.91,52.46,0.14,0.24,0.23,0.39
7,Выборг,233,0.88,58685.67,56.59,0.11,0.36,0.19,0.34
8,Петергоф,200,0.88,84838.85,50.99,0.16,0.30,0.19,0.35
9,Новое Девяткино,144,0.88,76136.76,52.28,0.15,0.30,0.21,0.34


### Анализ рынка недвижимости Ленинградской области

#### 1. Рейтинг населенных пунктов по количеству объявлений
Топ-15 населенных пунктов Ленобласти по публикациям:
- Мурино: 590
- Кудрово: 472
- Шушары: 440
- Всеволожск: 397
- Пушкин: 364
- Парголово: 326
- Гатчина: 306
- Колпино: 337
- Выборг: 233
- Петергоф: 200
- Сестрорецк: 182
- Красное Село: 176
- Новое Девяткино: 144
- Сертолово: 141
- Ломоносов: 133

#### 2. Доля проданной недвижимости по населенным пунктам
Во всех топ-15 населенных пунктах доля продаж превышает 83%. Наибольшие показатели демонстрируют:
- Мурино: 93%
- Кудрово: 93% 
- Шушары: 93%

#### 3. Стоимость и площадь недвижимости
Наибольшая стоимость квадратного метра:
- Пушкин: 103122.54
- Сестрорецк: 101912.34
- Кудрово: 95324.93

Наименьшая стоимость квадратного метра:
- Выборг: 58685.67
- Всеволожск: 68719.29
- Гатчина: 68935.19
- Сертолово: 69451.31

Наименьшая площадь продаваемых квартир:
- Мурино: 44.10 м²
- Кудрово: 46.40 м²

#### 4. Скорость продаж по населенным пунктам
Быстрее всего продается недвижимость:
- Колпино: 22% в первый месяц
- Мурино: 21% в первый месяц
- Кудрово: 19% в первый месяц

Медленнее всего продается недвижимость:
- Выборг: 11% в первый месяц
- Всеволожск: 11% в первый месяц

<a class="anchor" id="4-bullet"></a>
### Общие выводы и рекомендации

#### Что продается лучше всего
- Наибольшим спросом пользуются квартиры малой площади (48-54 м²)
- Быстрее всего продаются 2-комнатные квартиры с 1 балконом на 3-5 этажах
- В Санкт-Петербурге наблюдается обратная зависимость между ценой за м² и скоростью продажи
  
#### Сезонность рынка
- Наибольшая активность наблюдается в осенние месяцы (октябрь-ноябрь)
- Пики публикаций и продаж совпадают, что свидетельствует о высокой ликвидности в этот период
- Ценовые колебания имеют сезонный характер: минимальные цены весной, максимальные - в начале осени
 
#### Выгодные сегменты недвижимости Ленинградской области
- Мурино, Кудрово и Шушары лидируют по объему предложения и доле продаж (93%)
- Наибольшая стоимость м² в Пушкине, Сестрорецке и Кудрово
- Наименьшая стоимость м² в Выборге, Всеволожске и Гатчине
- Самая высокая скорость продаж в Колпино, Мурино и Кудрово

#### Стратегия выхода на рынок
1. Оптимальный период для запуска проектов - август-сентябрь
2. Пик маркетинговой активности планировать на октябрь-ноябрь
3. Весенний период использовать для приобретения недвижимости с дисконтом

#### Приоритетные сегменты
1. Сфокусироваться на 2-комнатных квартирах площадью 45-55 м²
2. Развивать предложение в сегменте недорогой компактной недвижимости
3. В премиальном сегменте закладывать более длительные сроки экспозиции

#### Географические приоритеты
1. Первоочередное внимание - Мурино, Кудрово, Шушары (высокая ликвидность)
2. Развивать присутствие в Колпино и Пушкине (высокая скорость продаж)
3. Рассмотреть Выборг и Всеволожск для работы с бюджетным сегментом